<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Build-function-for-building-cascade" data-toc-modified-id="Build-function-for-building-cascade-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Build function for building cascade</a></span></li><li><span><a href="#Import-dataset" data-toc-modified-id="Import-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import dataset</a></span><ul class="toc-item"><li><span><a href="#Store-cascade-data" data-toc-modified-id="Store-cascade-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Store cascade data</a></span></li><li><span><a href="#Store-cascade-seed-data" data-toc-modified-id="Store-cascade-seed-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Store cascade seed data</a></span></li></ul></li><li><span><a href="#Model" data-toc-modified-id="Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model</a></span></li></ul></div>

In [1]:
import tensorflow as tf
import edward as ed
import numpy as np

from edward.models import Beta, Exponential, Categorical, Multinomial

import networkx as nx

# Build function for building cascade

In [347]:
def build_cascade(alpha, seed, T):
    # Store number of nodes
    n = alpha.shape[0]
    
    
    # Keep a list of infected nodes
    infected = [seed]

    # Initialize transmission times to be max time except for seed node
    transmission = tf.ones(n)*T
    transmission = tf.subtract(transmission,tf.one_hot(seed, n)*T)

    # Keep track of time since last infected
    latest_infected_time = 0.

    # Iterate over all nodes
    for i in range(n-1):

        # Calculate transmission times for the newest infected node
        latest_transmission = tf.add(latest_infected_time,Exponential(tf.gather_nd(alpha,[[infected[i]]])))

        # Calculate minimum times between this
        transmission = tf.minimum(transmission, latest_transmission)

        # Use to identify the lowest i nodes that have been infected. The latest on the list is the latest infected
        top_k = tf.nn.top_k(-transmission,k=i+2)

        # Store infected node and infected time    
        latest_infected = top_k.indices[0][i+1]
        latest_infected_time = -top_k.values[0][i+1]

        # Store infected node and infected time
        infected.append(latest_infected)
        
    return transmission[0]

In [348]:
sess = ed.get_session()
alpha = tf.convert_to_tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],tf.float32)
sess.run(build_cascade(alpha, 0, 100))

array([  0.      ,   3.717908, 100.      , 100.      , 100.      ,
       100.      , 100.      , 100.      , 100.      , 100.      ],
      dtype=float32)

# Import dataset

In [160]:
with open('kronecker-core-periphery-n1024-h10-r0_01-0_25-1000-cascades.txt','r') as f:
    
    # Store number of nodes
    numNodes = -1
    while True:
        if f.readline() == "\n":
            break
        numNodes+=1

    # Collect cascades into list
    v = []
    for line in f.readlines():
        v.append([float(l) for l in line.rstrip('\n').split(",")])

## Store cascade data

In [338]:
np_cascades = np.ones((len(v),numNodes),np.float32)*10
for row, cascade in enumerate(v):  
    c_nodes = [int(cascade[i*2]) for i in range(len(cascade)//2)]
    c_times = [cascade[i*2+1] for i in range(len(cascade)//2)]

    for col in range(len(c_nodes)):
        np_cascades[row][c_nodes[col]]=c_times[col]

## Store cascade seed data

In [339]:
np_seeds = np.array([cascade[0] for cascade in v], dtype = np.int)

# Model

In [340]:
T = 10
ed_a = Beta(tf.ones([numNodes,numNodes])/2,tf.ones([numNodes,numNodes]))
seed_ph = Categorical(probs=np.ones(numNodes)/numNodes)
cascade_ph = build_cascade(ed_a, 753, T)

In [341]:
test = sess.run(cascade_ph)
print(test.shape)
print(np_cascades.shape)

(1023,)
(1001, 1023)


In [342]:
data = {cascade_ph: np_cascades[0]}
inference = ed.MAP([ed_a], data)

/Users/brianallen131/src/edward/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


In [343]:
tf.global_variables_initializer().run()

inference.run(n_iter=1000)

1000/1000 [100%] ██████████████████████████████ Elapsed: 51s | Loss: -34304092.000


In [344]:
optimized_alpha = sess.run(inference.latent_vars)

In [345]:
# np.array(optimized_alpha)
list(optimized_alpha.values())[0][753].argsort()

array([406,  91,   1, ..., 369, 467, 202])

In [346]:
v[0]
# list(optimized_alpha.values())[0][753]
# np_cascades 

[753.0,
 0.0,
 261.0,
 1.35072,
 704.0,
 3.966973,
 145.0,
 4.897442,
 73.0,
 5.925426,
 286.0,
 6.630252,
 556.0,
 7.843829,
 80.0,
 8.141659,
 458.0,
 8.775306,
 8.0,
 8.788488,
 467.0,
 8.902549,
 554.0,
 8.908081,
 53.0,
 9.237536,
 1.0,
 9.451033,
 584.0,
 9.591078,
 843.0,
 9.602514,
 116.0,
 9.780086,
 130.0,
 9.81525,
 545.0,
 9.828353,
 579.0,
 9.970419,
 535.0,
 9.984903]